## Resetting the ID

First step, make sure you are on the right ID and the right middleware

In [ ]:
import os
import subprocess

os.environ['ROS_DOMAIN_ID']="0"
os.environ['RMW_IMPLEMENTATION']="rmw_cyclonedds_cpp"
#os.environ['RMW_IMPLEMENTATION'] = 'rmw_fastrtps_cpp'
!printenv | grep 'ROS'
!printenv | grep 'RMW'
#print(os.environ['FASTRTPS_DEFAULT_PROFILES_FILE'])

Now restart ros and then try the topic list

In [ ]:
!ros2 daemon stop

In [ ]:
!ros2 topic list

if that did not work - try running through all possible IDs.

In [ ]:
import time
for i in range(19):
    list = subprocess.run(['ros2','daemon','stop'],capture_output=True)
    os.environ['ROS_DOMAIN_ID']=str(i)
    print(os.environ['ROS_DOMAIN_ID'])
    time.sleep(0.1)
    list = subprocess.run(['ros2','topic','list'],capture_output=True)
    print(list.stdout.decode().split('\n')[0])

## Test all webpages

First, check all connected Creates, run this code (assuming it was recently updated).

In [ ]:
import os

with open('/home/ddeche01/irobot-status.txt') as file:
    creates = file.readlines()
file.close()

clients = []
for create in creates:
    if create:
        try:
            name = create.split('hostName\" : \"')[1].split('\",')[0]
            IP = 'http://'+create.split('hostIpV4\" : \"')[1].split('\",')[0]
            print(name, end=': ')
            #print(IP)
            print(IP + '/ros-config')
            #print(IP + '/wifi')
            clients.append(create)
        except:
            pass

Next get all the info and test the battery level

In [ ]:
import os, sys
import subprocess
import requests

for client in clients:
    if client:
        name = client.split('hostName\" : \"')[1].split('\",')[0]
        url = 'http://' + client.split('hostIpV4\" : \"')[1].split('\",')[0]
        print(name, end=': ')
        print(url)
        print('    Testing Version ',end='')
        r = requests.get(url+'/home').text
        reply = r.split('VERSION: ')[1].split('</pre>')[0]
        print ('- good - ' if  ('G.3' in reply) else '- BAD - ',end='')
        print(reply)
        print('    Testing ID ',end='')
        r = requests.get(url+'/ros-config').text
        reply = r.split('required value=\"')[1].split('\"')[0]
        #print ('- good - ' if  (int(reply) == int(ID)) else '- BAD - ',end='')
        print(reply)
        print('    Testing namespace ',end='')
        reply = r.split('name=\"ros_namespace" value=\"')[1].split('\"')[0]
        print ('- good - ' if  (reply) else '- BAD - ',end='')
        print(reply)

        print('    Testing middleware ',end='')
        option1 = r.split('<option value=\"')[1].split('</option>')[0]
        option2 = r.split('<option value=\"')[2].split('</option>')[0]
        reply='failed'
        reply = option1.split('\"')[0] if 'selected' in option1 else reply
        reply = option2.split('\"')[0] if 'selected' in option2 else reply
        #print ('- good - ' if  (mware in reply) else '- BAD - ',end='')
        print(reply)


And now for reading them all off the list and checking based on the logs

In [ ]:
import os
import time, requests
from Subs.CreateLib import Create

with open('/home/ddeche01/irobot-status.txt') as file:
    creates = file.readlines()
file.close()

clients = []
for create in creates:
    if create:
        try:
            name = create.split('hostName\" : \"')[1].split('\",')[0]
            IP = 'http://'+create.split('hostIpV4\" : \"')[1].split('\",')[0]
            #r = requests.get(IP+'/ros-config').text
            #ID = r.split('required value=\"')[1].split('\"')[0]
            r = requests.get(IP+'/logs-raw').text
            RMW = r.split('RMW_IMPLEMENTATION=')[-1].split('\n')[0]
            name = r.split('ROS_NAMESPACE=')[-1].split('\n')[0]
            ID = r.split('ROS_DOMAIN_ID=')[-1].split('\n')[0]
            print('%s (%s): %s >> %s'%(name,ID,IP,RMW))
            clients.append([name,ID,IP,RMW])
        except:
            pass
        
for create in clients:
    os.environ['ROS_DOMAIN_ID']=create[1]
    MyCreate = Create(create[0])
    try:
        print(create[0],end=': ')
        print(MyCreate.battery())
        MyCreate.beep()
        MyCreate.close()
    except:
        pass